In [1]:
!pip install langchain_community
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 30.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 39.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 415.4/415.4 kB 26.4 MB/s eta 0:00:00
  Attempting uninstall: async-timeout
    Found existing installation: async-timeout 5.0.1
    Uninstalling async-timeout-5.0.1:
      Successfully uninstalled async-timeout-5.0.1
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.25
    Uninstalling langchain-core-0.3.25:
      Successfully uninstalled langchain-core-0.3.25
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.3
    Uninstalling langchain-text-splitters-0.3.3:
      Successfully uninstalled langchain-text-splitters-0.3.3
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.12
    Uninstalling langchain-0.3.12:
      Successfully uninstalled langchain-0.3.

In [2]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("GROQ_API_KEY")
secret_value_1 = user_secrets.get_secret("HUGGINGFACE_WEAVIATE")
secret_value_2 = user_secrets.get_secret("LANGCHAIN_API_KEY")

In [3]:
from langchain.text_splitter import TokenTextSplitter, SentenceTransformersTokenTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

with open("/kaggle/input/hochiminh/HoChiMinh.txt", "r", encoding="utf-8") as f:
    docs = f.read()
##1 - 0 - 1000 , 800 - 1800

# Split - Chunking
# Khởi tạo bộ chia văn bản
# Khởi tạo bộ chia văn bản
text_splitter = SentenceTransformersTokenTextSplitter(
    chunk_size=2000,  # Số token mỗi đoạn
    chunk_overlap=50,  # Overlap
    model_name="BAAI/bge-m3"  # Model tokenizer sử dụng
)

# Chia nhỏ văn bản
texts = text_splitter.create_documents([docs])

# Kiểm tra kết quả
print(f"Số đoạn sau khi chia: {len(texts)}")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/15.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/687 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

Số đoạn sau khi chia: 4


In [4]:
import time

# Embed
model_name = "BAAI/bge-m3"
model_kwargs = {"device": "cuda"}
encode_kwargs = {"normalize_embeddings": True}
hf_embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)

vectorstore = FAISS.from_documents(documents=texts, 
                                    embedding=hf_embeddings)

retriever = vectorstore.as_retriever() # Dense Retrieval - Embeddings/Context based

<ipython-input-4-814b61e38edf>:7: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  hf_embeddings = HuggingFaceBgeEmbeddings(


In [5]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

/usr/local/lib/python3.10/dist-packages/langsmith/client.py:261: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Tải tokenizer
tokenizer = AutoTokenizer.from_pretrained("vilm/vinallama-2.7b", trust_remote_code=True)

# Tải mô hình với device_map="auto" để dùng cả 2 GPU
model = AutoModelForCausalLM.from_pretrained(
    "vilm/vinallama-2.7b",
    device_map="auto",  # Phân bổ tự động qua các GPU
    trust_remote_code=True
)

print("Load mô hình thành công trên multi-GPU!")

tokenizer_config.json:   0%|          | 0.00/1.03k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.67M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/552 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/672 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/5.55G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

Load mô hình thành công trên multi-GPU!


In [2]:
import torch

# Kiểm tra VRAM
print(f"GPU 0 VRAM: {torch.cuda.memory_allocated(0) / 1024**3:.2f} GB")
print(f"GPU 1 VRAM: {torch.cuda.memory_allocated(1) / 1024**3:.2f} GB")

# Prompt
prompt = "Xin chào, bạn khỏe không?"
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
print("Bước 2")
# Sinh văn bản
outputs = model.generate(**inputs, max_length=200, temperature=0.1)
print("Bước 3")
response = tokenizer.decode(outputs[0], skip_special_tokens=True)

print("Kết quả:", response)

GPU 0 VRAM: 4.95 GB
GPU 1 VRAM: 5.56 GB
Bước 2


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


Bước 3
Kết quả: Xin chào, bạn khỏe không?
Chào bạn, bạn khỏe không?
Chào bạn, bạn khỏe không?
Chào bạn, bạn khỏe không?
Chào bạn, bạn khỏe không?
Chào bạn, bạn khỏe không?
Chào bạn, bạn khỏe không?
Chào bạn, bạn khỏe không?
Chào bạn, bạn khỏe không?
Chào bạn, bạn khỏe không?
Chào bạn, bạn khỏe không?
Chào bạn, bạn khỏe không?
Chào bạn, bạn khỏe không?
Chào bạn, bạn khỏe không?
Chào bạn, bạn khỏe không?
Chào bạn, bạn khỏe không?
Chào bạn, bạn khỏe không?
Chào bạn, bạn khỏe không?
Chào bạn, bạn khỏe không?
Chào bạn, bạn khỏe không?
Chào bạn, bạn khỏe không?
Chào bạn, bạn khỏe không?
Chào bạn, bạn khỏe không?
Chào bạn, bạn khỏe không?
Chào bạn, bạn khỏe không?


In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Tải tokenizer
tokenizer = AutoTokenizer.from_pretrained("vilm/vinallama-7b", trust_remote_code=True)

# Tải mô hình với device_map="auto" để dùng cả 2 GPU
model = AutoModelForCausalLM.from_pretrained(
    "vilm/vinallama-7b",
    device_map="auto",  # Phân bổ tự động qua các GPU
    trust_remote_code=True
)

print("Load mô hình thành công trên multi-GPU!")

tokenizer_config.json:   0%|          | 0.00/861 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.67M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/671 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.91G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.80G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

Load mô hình thành công trên multi-GPU!


In [ ]:
import torch

# Kiểm tra VRAM
print(f"GPU 0 VRAM: {torch.cuda.memory_allocated(0) / 1024**3:.2f} GB")
print(f"GPU 1 VRAM: {torch.cuda.memory_allocated(1) / 1024**3:.2f} GB")

# Prompt
prompt = "Xin chào, bạn khỏe không?"
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
print("Bước 2")
# Sinh văn bản
outputs = model.generate(**inputs, max_length=200, temperature=0.1)
print("Bước 3")
response = tokenizer.decode(outputs[0], skip_special_tokens=True)

print("Kết quả:", response)